# <font color = "red">Classificação de Risco de Crédito</font>

## MODELAGEM PREDITIVA (CLASSIFICAÇÃO)

<font color = "red">Problema de Negócio:</font> A partir de dados históricos, irei verificar os padrões que levaram os clientes de uma instituição financeira, a adentrarem, ou não, na situação de inadimplência, para posteriormente criar um 

<font color = "red">Meta de Acurácia Final:</font> ACC mínimo de 88%

<font color = "blue">Nesta etapa, realizarei o treinamento dos modelos de classificação, para posterior comparação e teste, a fim de verificar as métricas. 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier

import warnings
warnings.filterwarnings('ignore')

In [5]:
# Importação dos dados 
X = pd.read_csv("C:/Projetos Pessoais/DataScience/analise_Risco_Credito/data/X_pre_processado_5.csv")
y = pd.read_csv("C:/Projetos Pessoais/DataScience/analise_Risco_Credito/data/y_pre_processado_5.csv")

In [6]:
X

,Renda_Anual,Anos_no_Emprego,Grau_Emprestimo,Valor,Taxa_Juros,Comprometimento_Renda,Inadimplencia_Historica,Tipo_Moradia_Hipoteca,Tipo_Moradia_Outro,Tipo_Moradia_Próprio,Motivo_Emprestimo_Educação,Motivo_Emprestimo_Empreendimento,Motivo_Emprestimo_Médico,Motivo_Emprestimo_Pessoal,Motivo_Emprestimo_Reformas Domésticas
0,-1.903478,0.196962,1,-1.551342,0.074325,-0.663416,0,0,0,1,1,0,0,0,0
1,-1.891412,-0.699305,0,-1.227123,-1.184422,1.077414,0,0,0,1,0,1,0,0,0
2,-1.887390,0.495717,3,-1.421655,1.207196,0.032916,0,0,0,1,0,1,0,0,0
3,-1.855214,1.093228,1,-1.389233,0.027122,0.032916,0,1,0,0,1,0,0,0,0
4,-1.847975,-1.296815,0,-1.443269,-1.137219,-0.199194,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34937,0.709549,-0.416085,4,2.555442,2.053350,1.309524,0,0,0,0,0,0,0,0,0
34938,-1.533943,-0.699305,3,-0.761657,1.393679,1.015354,1,0,0,0,0,0,0,1,0
34939,0.874144,0.959136,2,-0.120001,0.275724,-0.687167,1,1,0,0,0,0,0,0,0
34940,-1.112140,-0.998060,1,0.318936,-0.121854,1.975202,0,0,0,0,1,0,0,0,0


In [7]:
# Divisão em Dados de Treino e Teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X,
                                                        y,
                                                        test_size = 0.2,
                                                        random_state = 40)

# <font color = 'red'> --------------

# Regressão Logística 

In [8]:
modelo_v1_regressao = LogisticRegression()

In [9]:
modelo_v1_regressao.fit(X_treino, y_treino)

LogisticRegression()

In [10]:
previsoes_v1_LR = modelo_v1_regressao.predict(X_teste)

In [11]:
modelo_v1_metricas = {
 "Modelo": "Regressão Logística",
 "Versão": "1",
 "Detalhes": " - ",
 "Precision": round(precision_score(previsoes_v1_LR, y_teste), 3),
 "Recall": round(recall_score(previsoes_v1_LR, y_teste), 3),
 "F1_Score": round(f1_score(previsoes_v1_LR, y_teste), 3),
 "Acurácia": round(accuracy_score(previsoes_v1_LR, y_teste), 3),
 "AUC": round(roc_auc_score(y_teste, previsoes_v1_LR), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}

modelo_v1_metricas

{'Modelo': 'Regressão Logística',
 'Versão': '1',
 'Detalhes': ' - ',
 'Precision': 0.792,
 'Recall': 0.795,
 'F1_Score': 0.794,
 'Acurácia': 0.792,
 'AUC': 0.792}

# <font color = 'red'> --------------</font>

# <font color = 'red'>Decision Tree com Grid Search</font>

In [12]:
# Parametros para grid
params_decision_tree = {
 "criterion": ["gini", "entropy"],
 "max_depth": [3, None],
 "min_samples_leaf": [1, 3, 10],
 "min_samples_split": [2, 3],
 "max_depth": [1,3,10]
}

In [13]:
clf_decision_tree = DecisionTreeClassifier()

In [14]:
# Grid Search
modelo_v2_decisionTree = GridSearchCV(clf_decision_tree,
                                      params_decision_tree,
                                      scoring='accuracy',
                                      cv = 5)

In [15]:
# Treinando 
modelo_v2_decisionTree.fit(X_treino, y_treino)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 3, 10],
                         'min_samples_leaf': [1, 3, 10],
                         'min_samples_split': [2, 3]},
             scoring='accuracy')

In [16]:
modelo_v2_decisionTree.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 3}

In [17]:
previsoes_v2_DT = modelo_v2_decisionTree.predict(X_teste)

In [18]:
modelo_v2_metricas = {
 "Modelo": "Decision Tree GS",
 "Versão": "2",
 "Detalhes": "gini",
 "Precision": round(precision_score(previsoes_v2_DT, y_teste), 3),
 "Recall": round(recall_score(previsoes_v2_DT, y_teste), 3),
 "F1_Score": round(f1_score(previsoes_v2_DT, y_teste), 3),
 "Acurácia": round(accuracy_score(previsoes_v2_DT, y_teste), 3),
 "AUC": round(roc_auc_score(y_teste, previsoes_v2_DT), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}
modelo_v2_metricas

{'Modelo': 'Decision Tree GS',
 'Versão': '2',
 'Detalhes': 'gini',
 'Precision': 0.706,
 'Recall': 0.972,
 'F1_Score': 0.817,
 'Acurácia': 0.841,
 'AUC': 0.842}

# <font color = 'red'> --------------</font>

# <font color = 'red'>Random Forest sem Grid Search</font>

In [19]:
clf_random_forest = RandomForestClassifier(n_estimators = 100)

In [20]:
clf_random_forest.fit(X_treino, y_treino)

RandomForestClassifier()

In [22]:
previsoes_v3_RF = clf_random_forest.predict(X_teste)

In [24]:
modelo_v3_metricas = {
 "Modelo": "Random Forest",
 "Versão": "3",
 "Detalhes": "Sem otimização",
 "Precision": round(precision_score(previsoes_v3_RF, y_teste), 3),
 "Recall": round(recall_score(previsoes_v3_RF, y_teste), 3),
 "F1_Score": round(f1_score(previsoes_v3_RF, y_teste), 3),
 "Acurácia": round(accuracy_score(previsoes_v3_RF, y_teste), 3),
 "AUC": round(roc_auc_score(y_teste, previsoes_v3_RF), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}
modelo_v3_metricas

{'Modelo': 'Random Forest',
 'Versão': '3',
 'Detalhes': 'Sem otimização',
 'Precision': 0.892,
 'Recall': 0.963,
 'F1_Score': 0.926,
 'Acurácia': 0.928,
 'AUC': 0.929}

# <font color = 'red'> --------------</font>

# <font color = 'red'>Random Forest com GS</font>

In [25]:
clf_random_forest_GS = RandomForestClassifier(n_estimators = 100)

In [26]:
# Parametros
params_random_forest = {
    'max_depth': [None, 2],  # Profundidade máxima de cada árvore
    'min_samples_split': [2, 5],  # Mínimo de amostras necessárias para dividir um nó
    'min_samples_leaf': [1, 2, 3],    # Mínimo de amostras necessárias em um nó folha
    'criterion': ['gini', 'entropy']
}

In [33]:
# Grid Search
modelo_v4_randomForest_GS = GridSearchCV(clf_random_forest_GS,
                                      params_random_forest,
                                      cv = 5)

In [34]:
modelo_v4_randomForest_GS.fit(X_treino, y_treino)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 2], 'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 5]})

In [35]:
modelo_v4_randomForest_GS.best_params_

{'criterion': 'entropy',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [36]:
# Previsões do modelo 4 
previsoes_v4_RF_GS = modelo_v4_randomForest_GS.predict(X_teste)

In [37]:
modelo_v4_metricas = {
 "Modelo": "Random Forest GS",
 "Versão": "4",
 "Detalhes": "entropy",
 "Precision": round(precision_score(previsoes_v4_RF_GS, y_teste), 3),
 "Recall": round(recall_score(previsoes_v4_RF_GS, y_teste), 3),
 "F1_Score": round(f1_score(previsoes_v4_RF_GS, y_teste), 3),
 "Acurácia": round(accuracy_score(previsoes_v4_RF_GS, y_teste), 3),
 "AUC": round(roc_auc_score(y_teste, previsoes_v4_RF_GS), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}
modelo_v4_metricas

{'Modelo': 'Random Forest GS',
 'Versão': '4',
 'Detalhes': 'entropy',
 'Precision': 0.892,
 'Recall': 0.96,
 'F1_Score': 0.925,
 'Acurácia': 0.927,
 'AUC': 0.927}

# <font color = 'red'> --------------</font>

# <font color = 'red'>Extra Trees</font>

In [38]:
clf_exta_trees = ExtraTreesClassifier(n_estimators = 100)

In [39]:
# Parametros
params_extra_trees = {
    'max_depth': [None, 2],  # Profundidade máxima de cada árvore
    'min_samples_split': [2, 3, 5],  # Mínimo de amostras necessárias para dividir um nó
    'min_samples_leaf': [1, 2, 3],    # Mínimo de amostras necessárias em um nó folha
    'criterion': ['gini', 'entropy']
}

# Grid Search
modelo_v5_extraTrees_GS = GridSearchCV(clf_exta_trees,
                                         params_random_forest,
                                         cv = 5)

In [41]:
modelo_v5_extraTrees_GS.fit(X_treino, y_treino)

modelo_v5_extraTrees_GS.best_params_

{'criterion': 'entropy',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [42]:
# Previsões do modelo 5
previsoes_v5_extraTrees_GS = modelo_v5_extraTrees_GS.predict(X_teste)

In [44]:
modelo_v5_metricas = {
 "Modelo": "Extra Trees",
 "Versão": "5",
 "Detalhes": "entropy",
 "Precision": round(precision_score(previsoes_v5_extraTrees_GS, y_teste), 3),
 "Recall": round(recall_score(previsoes_v5_extraTrees_GS, y_teste), 3),
 "F1_Score": round(f1_score(previsoes_v5_extraTrees_GS, y_teste), 3),
 "Acurácia": round(accuracy_score(previsoes_v5_extraTrees_GS, y_teste), 3),
 "AUC": round(roc_auc_score(y_teste, previsoes_v5_extraTrees_GS), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}
modelo_v5_metricas

{'Modelo': 'Extra Trees',
 'Versão': '5',
 'Detalhes': 'entropy',
 'Precision': 0.906,
 'Recall': 0.942,
 'F1_Score': 0.924,
 'Acurácia': 0.924,
 'AUC': 0.924}

# <font color = 'red'> --------------</font>

# <font color = 'red'>Adaboost</font>

In [45]:
clf_adaboost = AdaBoostClassifier()

In [62]:
# Estimador base (usarei a melhor árvore de decisão)
estimador_base = DecisionTreeClassifier(criterion = 'gini',
                                        max_depth = 10,
                                        min_samples_leaf = 1, 
                                        min_samples_split = 3)

In [68]:
modelo_v6_adaBoost = AdaBoostClassifier(base_estimator = estimador_base,
                                        learning_rate = 0.5,
                                        n_estimators = 300,
                                        algorithm='SAMME')

In [69]:
modelo_v6_adaBoost.fit(X_treino, y_treino)

AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(max_depth=10,
                                                         min_samples_split=3),
                   learning_rate=0.5, n_estimators=300)

In [70]:
modelo_v6_adaBoost.get_params()

{'algorithm': 'SAMME',
 'base_estimator__ccp_alpha': 0.0,
 'base_estimator__class_weight': None,
 'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 10,
 'base_estimator__max_features': None,
 'base_estimator__max_leaf_nodes': None,
 'base_estimator__min_impurity_decrease': 0.0,
 'base_estimator__min_samples_leaf': 1,
 'base_estimator__min_samples_split': 3,
 'base_estimator__min_weight_fraction_leaf': 0.0,
 'base_estimator__random_state': None,
 'base_estimator__splitter': 'best',
 'base_estimator': DecisionTreeClassifier(max_depth=10, min_samples_split=3),
 'estimator': None,
 'learning_rate': 0.5,
 'n_estimators': 300,
 'random_state': None}

In [71]:
previsoes_v6_adaBoost = modelo_v6_adaBoost.predict(X_teste)

In [72]:
modelo_v6_metricas = {
 "Modelo": "AdaBoost",
 "Versão": "6",
 "Detalhes": "DecisionTree - Gini",
 "Precision": round(precision_score(previsoes_v6_adaBoost, y_teste), 3),
 "Recall": round(recall_score(previsoes_v6_adaBoost, y_teste), 3),
 "F1_Score": round(f1_score(previsoes_v6_adaBoost, y_teste), 3),
 "Acurácia": round(accuracy_score(previsoes_v6_adaBoost, y_teste), 3),
 "AUC": round(roc_auc_score(y_teste, previsoes_v6_adaBoost), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}
modelo_v6_metricas

{'Modelo': 'AdaBoost',
 'Versão': '6',
 'Detalhes': 'DecisionTree - Gini',
 'Precision': 0.931,
 'Recall': 0.972,
 'F1_Score': 0.951,
 'Acurácia': 0.951,
 'AUC': 0.952}

# <font color = 'red'> --------------</font>

# <font color = 'red'>Support Vector Classifier</font>

In [92]:
clf_svc = SVC()

In [96]:
param = {
 'kernel' : ['Linear', 'rbf', 'poly']
}

In [97]:
C_range = np.array([50., 100., 200.]),
gamma_range = np.array([0.3*0.001, 0.001, 3*0.001])
kernel_svm = ['Linear', 'rbf', 'poly']

svm_param_grid = dict(kernel = kernel_svm, gamma = gamma_range, C = C_range)
modelo_v7_SVC_GS = GridSearchCV(clf_svc, param, cv = 3) # cv = validações cruzadas


In [98]:
modelo_v7_SVC_GS.fit(X_treino, y_treino)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'kernel': ['Linear', 'rbf', 'poly']})

In [102]:
modelo_v7_SVC_GS.best_params_

{'kernel': 'rbf'}

In [103]:
previsoes_v7_SVM_kernel_rbf = modelo_v7_SVC_GS.predict(X_teste)

In [104]:
modelo_v7_metricas = {
 "Modelo": "SVC",
 "Versão": "7",
 "Detalhes": "Kernel RBF",
 "Precision": round(precision_score(previsoes_v7_SVM_kernel_rbf, y_teste), 3),
 "Recall": round(recall_score(previsoes_v7_SVM_kernel_rbf, y_teste), 3),
 "F1_Score": round(f1_score(previsoes_v7_SVM_kernel_rbf, y_teste), 3),
 "Acurácia": round(accuracy_score(previsoes_v7_SVM_kernel_rbf, y_teste), 3),
 "AUC": round(roc_auc_score(y_teste, previsoes_v7_SVM_kernel_rbf), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}
modelo_v7_metricas

{'Modelo': 'SVC',
 'Versão': '7',
 'Detalhes': 'Kernel RBF',
 'Precision': 0.766,
 'Recall': 0.901,
 'F1_Score': 0.828,
 'Acurácia': 0.839,
 'AUC': 0.84}

## Final da modelagem preditiva

abaixo realizarei a comparação entre os modelos treinados 

In [105]:
resumo = pd.DataFrame({"Modelo_V1": pd.Series(modelo_v1_metricas),
                       "Modelo_V2": pd.Series(modelo_v2_metricas),
                       "Modelo_V3": pd.Series(modelo_v3_metricas),
                       "Modelo_V4": pd.Series(modelo_v4_metricas),
                       "Modelo_V5": pd.Series(modelo_v5_metricas),
                       "Modelo_V6": pd.Series(modelo_v6_metricas),
                       "Modelo_V7": pd.Series(modelo_v7_metricas)})

In [106]:
resumo

,Modelo_V1,Modelo_V2,Modelo_V3,Modelo_V4,Modelo_V5,Modelo_V6,Modelo_V7
Modelo,Regressão Logística,Decision Tree GS,Random Forest,Random Forest GS,Extra Trees,AdaBoost,SVC
Versão,1,2,3,4,5,6,7
Detalhes,-,gini,Sem otimização,entropy,entropy,DecisionTree - Gini,Kernel RBF
Precision,0.792,0.706,0.892,0.892,0.906,0.931,0.766
Recall,0.795,0.972,0.963,0.96,0.942,0.972,0.901
F1_Score,0.794,0.817,0.926,0.925,0.924,0.951,0.828
Acurácia,0.792,0.841,0.928,0.927,0.924,0.951,0.839
AUC,0.792,0.842,0.929,0.927,0.924,0.952,0.84


## Salvando Modelo com Melhor performance

Modelo_v6 que utiliza o AdaBoost 

In [117]:
import joblib
import os

In [116]:
directory = 'C:/Projetos Pessoais/DataScience/analise_Risco_Credito/models'
if not os.path.exists(directory):
    os.makedirs(directory)

# Salve o modelo no arquivo especificado
modelo_path = os.path.join(directory, 'modelo_v6_adaboost.pkl')
joblib.dump(modelo_v6_adaBoost, modelo_path)

['C:/Projetos Pessoais/DataScience/analise_Risco_Credito/models\\modelo_v6_adaboost.pkl']

# <font color = 'red'> --------------</font>

# <font color = 'red'>Deploy</font>

In [120]:
X_teste

,Renda_Anual,Anos_no_Emprego,Grau_Emprestimo,Valor,Taxa_Juros,Comprometimento_Renda,Inadimplencia_Historica,Tipo_Moradia_Hipoteca,Tipo_Moradia_Outro,Tipo_Moradia_Próprio,Motivo_Emprestimo_Educação,Motivo_Emprestimo_Empreendimento,Motivo_Emprestimo_Médico,Motivo_Emprestimo_Pessoal,Motivo_Emprestimo_Reformas Domésticas
17130,-0.359045,-1.296815,2,-0.254463,0.558942,-0.083139,0,0,0,0,0,0,0,0,1
14704,-0.278445,-0.699305,2,0.826269,0.807544,0.961358,1,0,0,0,0,0,1,0,0
25050,-0.966405,0.454315,3,-1.254081,1.338785,-0.959859,1,0,0,0,0,0,0,0,0
32517,-0.164683,1.748870,0,-0.077445,-1.620365,-0.102575,0,1,0,0,0,0,1,0,0
34727,0.518544,-1.296815,3,-0.792606,1.436555,-1.092773,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22598,-0.857761,-0.794564,0,-0.470610,-1.431340,0.159054,0,0,0,0,0,0,0,0,0
12987,0.083370,1.093228,1,1.907002,-0.205747,1.541635,0,0,0,0,0,1,0,0,0
32244,-1.011976,-0.688377,3,0.071074,1.623044,1.315184,0,0,0,0,0,0,1,0,0
12022,-1.565633,0.495717,2,-1.443269,0.536914,-0.895526,1,0,0,0,0,0,0,0,0


In [123]:
X_teste.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6989 entries, 17130 to 25347
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Renda_Anual                            6989 non-null   float64
 1   Anos_no_Emprego                        6989 non-null   float64
 2   Grau_Emprestimo                        6989 non-null   int64  
 3   Valor                                  6989 non-null   float64
 4   Taxa_Juros                             6989 non-null   float64
 5   Comprometimento_Renda                  6989 non-null   float64
 6   Inadimplencia_Historica                6989 non-null   int64  
 7   Tipo_Moradia_Hipoteca                  6989 non-null   int64  
 8   Tipo_Moradia_Outro                     6989 non-null   int64  
 9   Tipo_Moradia_Próprio                   6989 non-null   int64  
 10  Motivo_Emprestimo_Educação             6989 non-null   int64  
 11  Moti